In [ ]:
import dask
import datetime
import matplotlib.pyplot as plt
import xarray as xr

from pathlib import PurePath, Path
from snobedo.lib.dask_utils import start_cluster, client_ip_and_port

from common import SNOBAL_DIR, DATA_DIR, SHARED_STORE

from palettable.cartocolors.sequential import Teal_7_r as TealCMP

In [ ]:
water_year = 'wy2021'

In [ ]:
def plot_day(snow_day, plot_opts, title):
    plt.figure(figsize=(10, 10), dpi=300, tight_layout=True)
    snow_day.plot(**plot_opts)
    plt.xlabel('')
    plt.ylabel('')
    plt.gca().ticklabel_format(style='plain')
    plt.title(snow_day.time.dt.strftime('%Y-%m-%d').values)
    plt.savefig(
        f'{DATA_DIR}/snow_movie/{title}_{str(snow_day.time.dt.strftime("%y%m%d").values)}.png'
    )
    plt.close()

In [ ]:
def plot_thickness():
    snow = xr.open_mfdataset(
        f'{SNOBAL_DIR}/{water_year}/erw_hrrr_solar_modis/run*/snow.nc',
        parallel=True, chunks={'time': -1},
        preprocess=lambda ds: ds['thickness']
    )

    plot_opts = dict(
        cmap=plt.cm.Blues,
        vmin=0, vmax=3, 
        cbar_kwargs={'label': 'Snow Height'}
    )
    
    return [ 
        dask.delayed(plot_day)(day, plot_opts, 'snow_depth/snow_depth')
        for day in snow.thickness
    ]
        
def plot_albedo():
    snow = xr.open_mfdataset(
        f'{SHARED_STORE}/SMESHR/net_HRRR_MODIS/{water_year}/net_HRRR.*.nc',
        parallel=True, chunks={'time': 1},
        preprocess=lambda ds: ds['albedo']
    ).sel(time=datetime.time(11))

    plot_opts = dict(
        cmap=TealCMP.mpl_colormap,
        vmin=0, vmax=10000, 
        cbar_kwargs={'label': 'Albedo'}
    )
    
    return [ 
        dask.delayed(plot_day)(day, plot_opts, 'albedo/albedo')
        for day in snow.albedo
    ]


In [ ]:
client = start_cluster(8, 24)
client_ip_and_port(client)

In [ ]:
dask.compute(plot_albedo());

In [ ]:
client.shutdown()

### Convert to video using ffmpeg

```shell
ffmpeg -pattern_type glob -i '*.jpg' -crf 18 -filter:v "setpts=2.5*PTS" snow_depth.mp4

ffmpeg -framerate 12 -pattern_type glob -i '*.png' -c:v libx264 -vf "format=yuv420p" wy2021_snow_depth.mp4
```